# Group Details

## Group Name:

### Student 1:

### Student 2:

### Student 3:

# Loading Data and Preliminaries

In [33]:
%cd /content/drive/MyDrive/TUe/DeepLearning/Assignment2

[WinError 3] The system cannot find the path specified: '/content/drive/MyDrive/TUe/DeepLearning/Assignment2'
d:\Master\Y1\Deep learning\Deep-learning\assignment_2


In [34]:
import os

import matplotlib
import matplotlib.pyplot as plt

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm
from glob import glob
from sklearn.model_selection import train_test_split

In [35]:
def load_array(filename, task):
    datapoint = np.load(filename)
    if task == 'task 1':
        initial_state = datapoint['initial_state']
        terminal_state = datapoint['terminal_state']
        return initial_state, terminal_state
    elif task == 'task 2' or task == 'task 3':
        whole_trajectory = datapoint['trajectory']
        # change shape: (num_bodies, attributes, time) ->  num_bodies, time, attributes
        whole_trajectory = np.swapaxes(whole_trajectory, 1, 2)
        initial_state = whole_trajectory[:, 0]
        target = whole_trajectory[:, 1:, 1:]  # drop the first timepoint (second dim) and mass (last dim) for the prediction task
        return initial_state, target
    else:
        raise NotImplementedError("'task' argument should be 'task 1', 'task 2' or 'task 3'!")


# Data Handling and Preprocessing

In [36]:
def pad_array(data):
    # Pad the array with zeros if necessary to have 9 rows
    padded_input_data  = np.pad(data[0], ((0, 9 - data[0].shape[0]), (0, 0)), mode='constant')
    padded_target_data = np.pad(data[1], ((0, 9 - data[1].shape[0]), (0, 0)), mode='constant')
    return torch.tensor(padded_input_data, dtype=torch.float32), torch.tensor(padded_target_data, dtype=torch.float32)

def create_mask(data,padded_input_data):
    # Create a boolean mask array indicating the padded rows
    mask = np.ones_like(padded_input_data, dtype=bool)
    mask[data[0].shape[0]:] = False
    return torch.tensor(mask, dtype=torch.bool)

def pair_values(data):
    # Pair up the values of the same columns from every row with every other row
    num_rows, num_cols = data.shape
    padded_input_data = torch.zeros((num_rows, num_rows, num_cols, 2))

    for i in range(num_rows):
        pair_idx = 0
        for j in range(num_rows):

            padded_input_data[i, pair_idx, :, 0] = data[i]
            padded_input_data[i, pair_idx, :, 1] = data[j]
            pair_idx += 1

    return padded_input_data

def get_euclidean_distance(x):
    euclidean_distances = torch.zeros((len(x), len(x)))
    for i in range(len(x)):
        source_x, source_y= x[i][1], x[i][2]
        #euclidean_distance=[]
        for j in range(len(x)):
            target_x, target_y= x[j][1], x[j][2]
            euclidean_distances[i][j]=np.sqrt((source_x-target_x)**2+(source_y-target_y)**2)
            #euclidean_distance.append(np.sqrt((source_x-target_x)**2+(source_y-target_y)**2))
        #euclidean_distances.append(euclidean_distance)

    return euclidean_distances #list of lists

def target_difference(target_x, source_x):
    # target x and source x are of shape (9, 2)
    #Subtract the x and y coordinates of the target from the source
    return target_x - source_x


def process_file(file_path):
    # Main function to process a single file
    data = load_array(file_path, 'task 1')
    padded_input_data, padded_target_data = pad_array(data)
    mask = create_mask(data,padded_input_data)
    input_data = pair_values(padded_input_data)
    euclidean_distance = get_euclidean_distance(padded_input_data)
    target_data= target_difference(padded_target_data, padded_input_data[:,1:3])

    return input_data, target_data, padded_input_data, mask, euclidean_distance


In [37]:
class CustomDataset_2(Dataset):
    def __init__(self, folder):
        self.folder = folder
        self.file_list = os.listdir(folder)

    def __len__(self):
        return len(self.file_list)

    def _read_file(self, file_path):
        input_data, target_data,data, mask, distances= process_file(file_path)
        return input_data, target_data,data, mask, distances

    def __getitem__(self, index):
        file_path = os.path.join(self.folder, self.file_list[index])
        # Read and preprocess the data from the file
        input_data, target_data,data, mask, distance = self._read_file(file_path)
        # Return the preprocessed data
        return input_data, target_data,data, mask, distance

# Model Implementation

## NPE implementation

In [38]:
class NPEEncoder(nn.Module):
    def __init__(self, hidden_units):
        super(NPEEncoder, self).__init__()
        self.pairwise_layer = nn.Linear(10, hidden_units, bias=False)
        self.feedforward = nn.Sequential(
            nn.Linear(hidden_units, 50, bias=False),
            nn.ReLU(),
            nn.Linear(50, 50, bias=False),
            nn.ReLU(),
            nn.Linear(50, 50, bias=False),
            nn.ReLU(),
            nn.Linear(50, 50, bias=False),
            nn.ReLU()
        )

    def forward(self, x):
        x= x.to(torch.float32)
        x = self.pairwise_layer(x)
        x = self.feedforward(x)
        return x

class NPEDecoder(nn.Module):
    def __init__(self):
        super(NPEDecoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.Linear(55, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 2)
        )

    def forward(self, x):
        x = self.decoder(x)
        return x

In [39]:
class NPEModel(nn.Module):
    def __init__(self, hidden_units, device):
        super(NPEModel, self).__init__()
        self.encoder = NPEEncoder(hidden_units)
        self.decoder = NPEDecoder()
        self.device = device

    def forward(self, x,unpaired_data, mask, distance, neighborhood_threshold=5):
        decoder_input = []
        dist_mask = ((distance > 0) & (distance < neighborhood_threshold))
        for batch in range(x.size(0)):
            batch_data= x[batch]
            for body in range(batch_data.size(0)):
                if torch.all(mask[batch][body]==1):
                    focus_body = unpaired_data[batch][body]

                    if dist_mask[batch][body].any():
                        chunk = torch.flatten(x[batch][body][dist_mask[batch][body]],start_dim=1)
                        neighbor_encodings = torch.sum(self.encoder(chunk), dim=0)
                    else:
                        neighbor_encodings = torch.zeros(50).to(self.device)
                    # Concatenate the focus body encoding with the focus_body
                    decoder_input.append(torch.cat((neighbor_encodings, focus_body), dim=0))
                else:
                    decoder_input.append(torch.cat((torch.zeros(50, device=self.device), torch.zeros_like(unpaired_data[batch][body])), dim=0))

        # Decode the concatenated vector
        decoded = self.decoder(torch.stack(decoder_input))

        out_shape = x.size()
        decoded = decoded.view(out_shape[0], out_shape[1], -1)

        return decoded

# Model Training

## NPE training

In [40]:
def load_data(folder):
    file_list = os.listdir(folder)
    output = []

    for filename in file_list:
        file_path = os.path.join(folder, filename)
        output.append(process_file(file_path))

    return output

In [123]:
data_dir = 'data/task 1/train'
# dataset = CustomDataset_2(data_dir)

dataset = load_data(data_dir)
train_dataset, valid_dataset = train_test_split(dataset, train_size=0.7) 


train_data_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_data_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

In [124]:
test_data_dir = 'data/task 1/test'
# test_dataset = CustomDataset_2(test_data_dir)

test_dataset = load_data(test_data_dir)


test_data_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [125]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
device

device(type='cpu')

In [126]:
# Hyperparameters
hidden_units = 25
learning_rate = 1e-3
learning_rate_decay = 0.99
iterations = 50
batch_size = 100

# Create the model
model = NPEModel(hidden_units, device=device)
model.to(device)
criterion = nn.MSELoss(reduction='none')
# optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [127]:
def train_sim_model(model, train_data_loader, valid_data_loader, optimizer, criterion,
                    device=torch.device('cpu'), iterations=50, model_file='best_model.pth'):
    best_val_loss = np.inf
    for iteration in tqdm(range(iterations)):
        model.train()
        train_loss=0.0
        for input_data, target_data,unpaired_data, mask, distance in train_data_loader:
        # Training steps
            input_data = input_data.to(device)
            unpaired_data = unpaired_data.to(device)
            mask = mask.to(device)
            distance = distance.to(device)
            target_data=target_data.to(device)

            optimizer.zero_grad()
            output = model(input_data, unpaired_data, mask, distance)

            loss = torch.masked_select(criterion(target_data, output),
                                       torch.stack([mask.all(dim=2), mask.all(dim=2)], dim=2)).mean()
            train_loss += loss.item()
            loss.backward()
            optimizer.step()

        train_loss /= len(train_data_loader)


        model.eval()
        test_loss=0.0
        with torch.no_grad():
            for input_data, target_data,unpaired_data, mask, distance in valid_data_loader:
                input_data = input_data.to(device)
                target_data = target_data.to(device)
                unpaired_data = unpaired_data.to(device)
                mask = mask.to(device)
                distance = distance.to(device)

                output = model(input_data, unpaired_data, mask, distance)
                loss = torch.masked_select(criterion(target_data, output),
                                       torch.stack([mask.all(dim=2), mask.all(dim=2)], dim=2)).mean()
                test_loss += loss.item()
        test_loss /= len(valid_data_loader)

        if test_loss < best_val_loss:
            best_val_loss = test_loss
            torch.save({
                'epoch': iteration+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, model_file)


            # Print loss for monitoring
            #if (iteration + 1) % 10000 == 0:
        print(f"Epoch [{iteration+1}/{iterations}], Loss: {train_loss}, \tTest Loss: {test_loss}")

In [128]:
train_sim_model(model, data_loader, test_data_loader, optimizer, criterion, device=device, iterations=iterations)

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch [1/50], Loss: 13.590798695882162, 	Test Loss: 10.983357429504395
Epoch [2/50], Loss: 12.265566380818685, 	Test Loss: 8.056063652038574
Epoch [3/50], Loss: 9.678956031799316, 	Test Loss: 6.868154525756836
Epoch [4/50], Loss: 8.900226656595866, 	Test Loss: 6.068338394165039
Epoch [5/50], Loss: 8.069623692830403, 	Test Loss: 5.753835439682007
Epoch [6/50], Loss: 7.712445672353109, 	Test Loss: 5.394040822982788
Epoch [7/50], Loss: 7.419311396280924, 	Test Loss: 5.180417060852051
Epoch [8/50], Loss: 7.384337647755941, 	Test Loss: 4.928468704223633
Epoch [9/50], Loss: 7.080880896250407, 	Test Loss: 4.635223865509033
Epoch [10/50], Loss: 6.466489315032959, 	Test Loss: 4.757339954376221
Epoch [11/50], Loss: 6.309070905049642, 	Test Loss: 4.37333083152771
Epoch [12/50], Loss: 6.274490769704183, 	Test Loss: 4.156847238540649
Epoch [13/50], Loss: 6.078954887390137, 	Test Loss: 3.8987756967544556
Epoch [14/50], Loss: 5.689714336395264, 	Test Loss: 3.893843173980713
Epoch [15/50], Loss: 5.602

## Evaluation task 1

In [129]:
def get_MSE(data_loader,model, files):
    MSE_total = 0
    MSE_data = []

    model.eval()
    with torch.no_grad():
        for input_data, target_diff, unpaired_data, mask, distance in data_loader:
            
                
                predicted_change = model(input_data, unpaired_data, mask, distance)

                mask_stack = torch.stack([mask.all(dim=2), mask.all(dim=2)], dim=2)

                MSE_data_batch = criterion(target_diff, predicted_change) * mask_stack.float()
                
                MSE_total += torch.masked_select(criterion(target_diff, predicted_change),
                                        torch.stack([mask.all(dim=2), mask.all(dim=2)], dim=2)).mean() * len(unpaired_data)/len(files)
            
                
                MSE_data.append([MSE_data_batch, unpaired_data, target_diff])

    return MSE_total, MSE_data

In [134]:
valid_MSE, valid_MSE_data = get_MSE(valid_data_loader,model, valid_files)

In [135]:
test_MSE, test_MSE_data = get_MSE(test_data_loader, model, test_files)

In [136]:
train_MSE, train_MSE_data = get_MSE(train_data_loader, model, train_files)

In [174]:
first_batch_MSE = valid_MSE_data[0]
first_batch_MSE = [[val.mean() for val in batch] for batch in first_batch_MSE[0]]
first_batch_MSE = [torch.stack(val).mean() for val in first_batch_MSE]

In [175]:
sorted_indexes = torch.argsort(torch.stack(first_batch_MSE))
top_3 = sorted_indexes[:3]
min_3 = sorted_indexes[-3:]


In [177]:
print("Indexes of the three lowest MSE scores:", top_3)
print("Indexes of the three highest MSE scores:", min_3)

Indexes of the three lowest MSE scores: tensor([ 3, 13, 33])
Indexes of the three highest MSE scores: tensor([11, 48,  0])


In [71]:
def plot_failure(model,mse_sorted, test_data_loader):
    first



tensor(4.6739)

In [72]:
sorted_MSE = sorted(MSE_test, key=lambda x: x[0][0])


In [73]:
sorted_MSE

[[tensor([2.4697e+00, 7.9084e+00, 3.6371e+00, 2.1662e-01, 9.2139e-01, 2.6708e+00,
          2.0253e-02, 9.5701e-02, 1.3912e+00, 2.7544e+00, 2.8111e-01, 7.5871e+00,
          1.1702e-01, 8.2659e-02, 1.4324e+01, 4.3912e+01, 2.0228e+00, 8.8618e-01,
          3.2669e-01, 1.1659e+00, 8.3892e-01, 5.1182e+00, 6.0706e-01, 6.2068e+00,
          5.6248e-01, 3.2912e+00, 2.2737e-01, 4.9544e-01, 1.3318e+00, 1.6574e+00,
          2.8157e+00, 1.3130e+01, 3.1771e+00, 1.2290e-01, 5.6798e-02, 5.3847e-02,
          1.3020e+01, 8.5630e-01, 1.6114e+00, 8.9897e+00, 3.9638e-01, 1.5766e+01,
          1.5757e+00, 4.7797e+00, 2.0041e+00, 2.0091e+01, 4.5060e+00, 3.6027e+00,
          6.8260e+00, 3.8316e+00, 4.9612e+01, 2.5098e+01, 2.9068e+00, 3.2514e-01,
          2.9456e+00, 8.7092e-01, 2.3758e+00, 2.2029e-01, 1.2318e+00, 7.8441e+00,
          3.7698e-01, 1.8434e-01, 2.9763e+00, 4.4275e-01, 3.2643e-05, 1.6915e-01,
          3.3391e-02, 6.1256e-03, 1.0508e+00, 1.5653e-01, 3.8059e+00, 5.1360e-02,
          2.9716

# Task 2

In [15]:
def files_2_data(files):
    output = []
    for file_path in tqdm(files):
        datapoint = np.load(file_path)
        whole_trajectory = datapoint['trajectory']
        # change shape: (num_bodies, attributes, time) ->  num_bodies, time, attributes
        whole_trajectory = np.swapaxes(whole_trajectory, 1, 2)
        for i in range(whole_trajectory.shape[1] - 1):
            data = whole_trajectory[:, i], whole_trajectory[:, i + 1, 1:3]
            padded_input_data, padded_target_data = pad_array(data)
            mask = create_mask(data,padded_input_data)
            input_data = pair_values(padded_input_data)
            euclidean_distance = get_euclidean_distance(padded_input_data)
            target_data= target_difference(padded_target_data, padded_input_data[:,1:3])

            output.append((input_data, target_data, padded_input_data, mask, euclidean_distance))
    return output

In [16]:
files_rate = 1.0

train_files = glob('data/task 2_3/train/*')
train_files = train_files[:int(len(train_files) * files_rate)]

train_files, valid_files = train_test_split(train_files, train_size=0.7)

In [17]:
train_data = files_2_data(train_files)

valid_data = files_2_data(valid_files)

  0%|          | 0/630 [00:00<?, ?it/s]

  0%|          | 0/270 [00:00<?, ?it/s]

In [18]:
BATCH_SIZE = 256

train_data_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_data_loader = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False)

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [20]:
# Hyperparameters
hidden_units = 25
LR = 1e-3
EPOCHS = 20

# Create the model
model = NPEModel(hidden_units, device=device).to(device)

criterion = nn.MSELoss(reduction='none')
optimizer = optim.AdamW(model.parameters(), lr=LR)

In [21]:
train_sim_model(model, train_data_loader, valid_data_loader, optimizer, criterion, device, iterations=1, model_file='best_task2_model.pth')

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch [1/1], Loss: 0.00473012076017316, 	Test Loss: 0.0015965685217032353


In [22]:
def recalc_data(padded_input_data):
    input_data = []
    euclidean_distance = []
    for elem in padded_input_data:
        input_data.append(pair_values(elem))
        euclidean_distance.append(get_euclidean_distance(elem))

    return torch.stack(input_data), padded_input_data, torch.stack(euclidean_distance)

In [23]:
def simulate(init_data, times=5):
    input_data, unpaired_data, mask, distance = init_data

    input_data = input_data.to(device)
    unpaired_data = unpaired_data.to(device)
    mask = mask.to(device)
    distance = distance.to(device)

    locations = []
    # locations = locations.to(device)
    model.eval()
    with torch.no_grad():
        for time in range(times):
            
            output = model(input_data, unpaired_data, mask, distance)
        
            unpaired_data[:,:,3:5] = output
            unpaired_data[:,:,1:3] += unpaired_data[:,:,3:5]
            locations.append(unpaired_data[:,:,1:3][0].clone())
            
           
            input_data, unpaired_data, distance = recalc_data(unpaired_data)

    return locations

# Evaluation

In [24]:
def files_2_data_test(files):
    output = []
    for file_path in tqdm(files):
        _, target = load_array(file_path, 'task 2')
        output.append((np.transpose(target[:,:4,:2], axes=(1, 0, 2)).reshape(target[:,:4,:2].shape[1],-1, target[:,:4,:2].shape[2])))
    return output

In [25]:
test_files = glob('data/task 2_3/test/*')
test_data = files_2_data(test_files)

  0%|          | 0/100 [00:00<?, ?it/s]

In [26]:
BATCH_SIZE = 1

test_data_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)
train_data_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_data_loader = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False)

In [27]:
def eval_model_2(data_loader, targets):
    model.eval()
    test_loss=0.0
    count = 0
    outputs = []

    with torch.no_grad():
        for input_data, target_data, unpaired_data, mask, euclidean_distance in data_loader:
            
            if count % 49 == 0:
                unpaired_data[torch.isnan(unpaired_data)] = 0

                input_data[torch.isnan(input_data)] = 0
                
                system = input_data, unpaired_data, mask, euclidean_distance
        
                output = simulate(system)
                outputs.append(output)
                target_data = targets[int(count/49)]


                output = torch.stack(output)
                target_data = [torch.from_numpy(val) for val in target_data]
    
                target_data = torch.stack(target_data)
                target_padded = torch.tensor([np.pad(tensor, ((0, 9 - tensor.shape[0]), (0, 0)), mode='constant') for tensor in target_data])

                print(target_padded)
                print(output)
                break
                loss = torch.masked_select(criterion(target_padded, output),
                                    torch.stack([mask.all(dim=2), mask.all(dim=2)], dim=2)).mean()
            
                test_loss += loss.item()


            count += 1
    test_loss /= int(len(data_loader)/49)

    return test_loss

In [28]:
targets_test = files_2_data_test(test_files)
eval_model_2(test_data_loader, targets_test)

  0%|          | 0/100 [00:00<?, ?it/s]

tensor([[[-2.6894, -0.0828],
         [-4.2424,  7.2504],
         [ 2.5693, -4.9245],
         [-2.7368,  0.1516],
         [-4.2096, -6.0082],
         [ 2.6533, -4.6996],
         [-3.3348,  2.2385],
         [ 4.8935, -8.4710],
         [ 0.0000,  0.0000]],

        [[-2.7741,  0.2847],
         [-4.1090,  7.2313],
         [ 2.7098, -4.6662],
         [-2.7578,  0.0601],
         [-4.2326, -5.9787],
         [ 2.5107, -5.0261],
         [-3.1910,  2.2147],
         [ 4.8268, -8.5243],
         [ 0.0000,  0.0000]],

        [[-2.8234,  0.2399],
         [-3.9748,  7.2087],
         [ 2.8266, -4.6453],
         [-2.8173,  0.4017],
         [-4.2535, -5.9465],
         [ 2.3772, -5.2491],
         [-3.0436,  2.1741],
         [ 4.7576, -8.5743],
         [ 0.0000,  0.0000]],

        [[-2.8782,  0.4877],
         [-3.8399,  7.1825],
         [ 2.9020, -4.6812],
         [-2.8727,  0.4671],
         [-4.2723, -5.9118],
         [ 2.2606, -5.4481],
         [-2.8935,  2.1121],
        

C:\Users\20172048\AppData\Local\Temp\ipykernel_15176\1638549923.py:26: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  target_padded = torch.tensor([np.pad(tensor, ((0, 9 - tensor.shape[0]), (0, 0)), mode='constant') for tensor in target_data])


0.0

In [29]:
targets_valid = files_2_data_test(valid_files)
eval_model_2(valid_data_loader, targets_valid)

  0%|          | 0/270 [00:00<?, ?it/s]

tensor([[[ 6.4239, -5.3166],
         [-4.5192, -1.1962],
         [ 8.3286,  5.5881],
         [ 3.6221, -2.4099],
         [ 3.9970, -3.6502],
         [-6.8688, -1.0557],
         [-4.6761, -0.0843],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 6.3696, -5.3293],
         [-4.5996, -1.1162],
         [ 8.3983,  5.5134],
         [ 3.6218, -2.5888],
         [ 3.9300, -3.5934],
         [-6.7828, -0.9486],
         [-4.7400,  0.0443],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 6.3109, -5.3379],
         [-4.6908, -1.0195],
         [ 8.4671,  5.4377],
         [ 3.6270, -2.7853],
         [ 3.8514, -3.4946],
         [-6.6854, -0.8402],
         [-4.8085,  0.1456],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 6.2475, -5.3425],
         [-4.7944, -0.9057],
         [ 8.5351,  5.3610],
         [ 3.6448, -3.0201],
         [ 3.7434, -3.3013],
         [-6.5748, -0.7304],
         [-4.8832,  0.2187],
        

0.0

In [30]:
targets_train = files_2_data_test(train_files)
eval_model_2(train_data_loader, targets_train)

  0%|          | 0/630 [00:00<?, ?it/s]

tensor([[[ 2.3894, -1.3275],
         [-3.6968,  5.6365],
         [-3.9164,  5.7696],
         [-3.5637,  5.4677],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 2.3515, -1.3916],
         [-3.8106,  6.0944],
         [-3.7438,  5.4319],
         [-3.5392,  5.2969],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 2.3127, -1.4546],
         [-3.9194,  6.4033],
         [-3.3212,  5.0432],
         [-3.6338,  5.2644],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 2.2732, -1.5167],
         [-4.0144,  6.6654],
         [-3.1956,  4.8633],
         [-3.6015,  5.1713],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
        

0.0